In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [26]:
from IPython.display import Markdown, display

In [ ]:
MODEL = "gemini-2.5-flash"
DB_NAME = "vector_db"
load_dotenv(override=True)

In [17]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

In [18]:
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0
)

In [ ]:
retriever.invoke("What should be my strategy if I have low budget?")

In [20]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly club advisor who helps us what to do as a club.
You will be asked questions about financial strategy and player details.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [27]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    display(Markdown(response.content))

In [ ]:
answer_question("Critical attributes for fullbacks?", [])